# Corporate Hybrid Forecast Notebook ((Prophet vs ARIMA vs TBATS/ETS)) – v4


## 01 - Imports & Settings

In [1]:

"""
Capacity Forecast 2026 – Hybrid (Prophet / ARIMA / TBATS-ETS) – v5

What's included vs. your v4.2:
- Model on monthly rate per workday: forecast rate in log1p scale, then multiply by future workdays (reduces variance & MAPE)
- Blending by wMAPE (weights ~ 1/wMAPE) while still reporting sMAPE in the CV sheet
- Robust month-to-month smoothing using rolling Median ± K·MAD (prevents spikes like 43k)
- More conservative ARIMA grid (less overreactive)
- ETS with damped trend added as a candidate for blending (stable option)
- Keep: log-scale modelling + expm1_safe + dynamic caps, winsorization, adaptive/sanitized CV, top-down daily reconciliation, sheet names

All comments are in English per your preference.
"""

import os
import warnings
from typing import Optional, Dict, Tuple
import numpy as np
import pandas as pd

from statsmodels.tsa.statespace.sarimax import SARIMAX
from statsmodels.tsa.holtwinters import ExponentialSmoothing
try:
    from prophet import Prophet
except Exception:
    Prophet = None
try:
    from tbats import TBATS
except Exception:
    TBATS = None

warnings.filterwarnings("ignore")

# ==================== Configuration ====================

# Inputs (adjust if your file locations change)
INCOMING_SOURCE_PATH = r"C:\Users\pt3canro\Desktop\CAPACITY\input_model\Incoming_new.xlsx"  # Sheet 'Main'
INCOMING_SHEET = "Main"
DEPT_MAP_PATH = r"C:\Users\pt3canro\Desktop\CAPACITY\input_model\department.xlsx"
DEPT_MAP_SHEET = "map"
PRODUCTIVITY_PATH = r"C:\Users\pt3canro\Desktop\CAPACITY\input_model\productivity_agents.xlsx"

# Output
OUTPUT_XLSX = r"C:\Users\pt3canro\Desktop\CAPACITY\outputs\capacity_forecast_hybrid.xlsx"

# Horizons and switches
H_MONTHS = 12             # monthly forecast horizon
DAILY_HORIZON_DAYS = 90   # daily plan horizon
REPORT_START_MONTH = "2025-01"  # show historical Actuals from this month in capacity_error

# Top-down reconciliation for daily forecasts
USE_DAILY_FROM_MONTHLY = True

# Strong "hard clipping" at output is disabled (we now use dynamic caps inside expm1_safe)
ENABLE_FORECAST_CLIP = False
FORECAST_CLIP_MULTIPLIER = 2.5  # kept for reference if you ever want to re-enable

# Optional final growth guard (disabled by default; enable if needed)
APPLY_LOCAL_GROWTH_GUARD = False
MAX_GROWTH = 1.8   # +80% vs local ref
MIN_GROWTH = 0.5   # -50% vs local ref

# Organization-specific
WEEKLY_START_THU = True

# Fixed language shares
LANGUAGE_SHARES = {
    'English': 0.6435,
    'French': 0.0741,
    'German': 0.0860,
    'Italian': 0.0667,
    'Portuguese': 0.0162,
    'Spanish': 0.1135
}



## 02. Data loading

In [2]:
def load_incoming(path: str, sheet_name: Optional[str] = None) -> pd.DataFrame:
    """Load daily incoming volumes. Build ticket_total if needed."""
    if not os.path.exists(path):
        raise FileNotFoundError(f"Incoming file not found:\n{path}\n")
    ext = os.path.splitext(path)[1].lower()
    if ext in [".xlsx", ".xlsm", ".xls"]:
        if not sheet_name:
            raise ValueError("Excel file detected but no sheet_name provided (e.g., 'Main').")
        df = pd.read_excel(path, sheet_name=sheet_name, engine="openpyxl")
    elif ext == ".csv":
        df = pd.read_csv(path)
    else:
        raise ValueError(f"Unsupported extension for incoming data: {ext}")

    base_required = {'Date', 'department_id'}
    missing = base_required - set(df.columns)
    if missing:
        raise ValueError(f"Incoming file must contain {sorted(list(base_required))}. Found: {list(df.columns)}")

    if 'ticket_total' not in df.columns:
        if 'total_incoming' in df.columns:
            df['ticket_total'] = pd.to_numeric(df['total_incoming'], errors='coerce').fillna(0)
        elif {'incoming_from_customers', 'incoming_from_transfers'}.issubset(df.columns):
            df['ticket_total'] = (
                pd.to_numeric(df['incoming_from_customers'], errors='coerce').fillna(0) +
                pd.to_numeric(df['incoming_from_transfers'], errors='coerce').fillna(0)
            )
        else:
            raise ValueError("Missing 'ticket_total' or components to create it.")

    df['Date'] = pd.to_datetime(df['Date'], errors='coerce')
    if df['Date'].isna().any():
        bad = df.loc[df['Date'].isna()]
        raise ValueError(f"Some Date values could not be parsed. Example rows:\n{bad.head(5)}")
    df['department_id'] = df['department_id'].astype(str).str.strip()
    df['ticket_total'] = pd.to_numeric(df['ticket_total'], errors='coerce').fillna(0).astype(float)

    if 'department_name' in df.columns:
        df['department_name'] = df['department_name'].astype(str).str.strip()
    else:
        df['department_name'] = None
    if 'vertical' in df.columns:
        df['vertical'] = df['vertical'].astype(str).str.strip()

    return df


def load_dept_map(path: str, sheet: Optional[str] = None) -> pd.DataFrame:
    """Load dept mapping -> department_name, vertical."""
    if not os.path.exists(path):
        return pd.DataFrame(columns=['department_id', 'department_name', 'vertical'])

    ext = os.path.splitext(path)[1].lower()
    if ext in (".xlsx", ".xlsm", ".xls"):
        if sheet:
            mp = pd.read_excel(path, sheet_name=sheet, engine="openpyxl")
        else:
            xls = pd.ExcelFile(path, engine="openpyxl")
            mp = pd.read_excel(xls, sheet_name=xls.sheet_names[0])
    else:
        mp = pd.read_csv(path)

    rename_map = {
        'dept_id': 'department_id',
        'dept_name': 'department_name',
        'name': 'department_name',
        'segment': 'vertical',
        'vertical_name': 'vertical'
    }
    mp = mp.rename(columns={k: v for k, v in rename_map.items() if k in mp.columns})
    if 'department_id' not in mp.columns:
        raise ValueError(f"Department map must contain 'department_id'. Found: {list(mp.columns)}")

    mp['department_id'] = mp['department_id'].astype(str).str.strip()
    mp['department_name'] = (mp['department_name'].astype(str).str.strip()
                             if 'department_name' in mp.columns else None)
    mp['vertical'] = (mp['vertical'].astype(str).str.strip()
                      if 'vertical' in mp.columns else None)

    return mp[['department_id', 'department_name', 'vertical']].drop_duplicates('department_id')


def load_productivity(path: str) -> pd.DataFrame:
    """Load agent productivity and compute dept-level mean tickets/agent-day."""
    if not os.path.exists(path):
        raise FileNotFoundError(f"Productivity file not found: {path}")
    df = pd.read_excel(path, engine="openpyxl")
    req = {'Date', 'agent_id', 'department_id', 'prod_total_model'}
    missing = req - set(df.columns)
    if missing:
        raise ValueError(f"productivity_agents.xlsx missing columns: {sorted(list(missing))}. Found: {list(df.columns)}")
    df['Date'] = pd.to_datetime(df['Date'], errors='coerce')
    df['department_id'] = df['department_id'].astype(str).str.strip()
    df['prod_total_model'] = pd.to_numeric(df['prod_total_model'], errors='coerce')

    prod_dept = (df.groupby('department_id', as_index=False)['prod_total_model']
                 .mean()
                 .rename(columns={'prod_total_model': 'avg_tickets_per_agent_day'}))
    return prod_dept

## 03. Helpers & metrics

In [3]:
def business_days_in_month(year: int, month: int) -> int:
    """Approximate Mon-Fri working days in a month."""
    rng = pd.date_range(start=pd.Timestamp(year=year, month=month, day=1),
                        end=pd.Timestamp(year=year, month=month, day=1) + pd.offsets.MonthEnd(0),
                        freq='D')
    return int(np.sum(rng.weekday < 5))


def smape(y_true, y_pred) -> float:
    """sMAPE robust for intermittent series."""
    y_true = np.array(y_true, dtype=float)
    y_pred = np.array(y_pred, dtype=float)
    denom = (np.abs(y_true) + np.abs(y_pred))
    denom[denom == 0] = 1.0
    return float(np.mean(2.0 * np.abs(y_pred - y_true) / denom) * 100.0)


def wmape(y_true, y_pred) -> float:
    """Weighted MAPE: sum(|e|)/sum(|y|)."""
    y_true = np.array(y_true, dtype=float)
    y_pred = np.array(y_pred, dtype=float)
    denom = np.sum(np.abs(y_true))
    if denom <= 0:
        return 200.0
    return float(100.0 * (np.sum(np.abs(y_true - y_pred)) / denom))


def apply_mapping(incoming: pd.DataFrame, mapping: pd.DataFrame) -> pd.DataFrame:
    """Merge department_name / vertical using department_id."""
    merged = incoming.merge(mapping, on='department_id', how='left', suffixes=('', '_map'))
    if 'department_name' not in merged.columns:
        merged['department_name'] = None
    if 'department_name_map' not in merged.columns:
        merged['department_name_map'] = None
    merged['department_name'] = merged['department_name'].fillna(merged['department_name_map']).fillna("Unknown")

    if 'vertical' not in merged.columns:
        merged['vertical'] = None
    if 'vertical_map' not in merged.columns:
        merged['vertical_map'] = None
    merged['vertical'] = merged['vertical'].fillna(merged['vertical_map']).fillna("Unmapped")

    drop_cols = [c for c in merged.columns if c.endswith('_map')]
    merged.drop(columns=drop_cols, inplace=True, errors='ignore')
    return merged


def winsorize_monthly(ts_m: pd.Series, lower_q: float = 0.01, upper_q: float = 0.99) -> pd.Series:
    """Winsorize monthly series to reduce the influence of extreme outliers."""
    if ts_m.empty:
        return ts_m
    lo = ts_m.quantile(lower_q)
    hi = ts_m.quantile(upper_q)
    return ts_m.clip(lower=lo, upper=hi)

# ---------- v4.2 safe inverse & dynamic cap ----------

def expm1_safe(log_vals: np.ndarray, cap_original: Optional[float] = None) -> np.ndarray:
    """
    Safe inverse of log1p:
    - replace non-finite logs by a very negative number (-> ~0)
    - lower-bound logs to avoid underflow
    - optional cap on original scale applied in log-domain and after expm1
    """
    x = np.array(log_vals, dtype=float)
    x[~np.isfinite(x)] = -50.0
    x = np.maximum(x, -50.0)

    if cap_original is not None and np.isfinite(cap_original) and cap_original > 0:
        log_cap = np.log1p(cap_original)
        x = np.minimum(x, log_cap)

    y = np.expm1(x)
    if cap_original is not None and np.isfinite(cap_original) and cap_original > 0:
        y = np.minimum(y, cap_original)
    return np.clip(y, 0, None)


def compute_dynamic_cap(ts_m: pd.Series) -> float:
    """Generous per-department cap on the original scale to prevent explosions."""
    if ts_m.empty or (ts_m.max() <= 0):
        return np.inf
    m12 = float(ts_m.tail(12).mean()) if len(ts_m) >= 3 else float(ts_m.mean())
    med = float(ts_m.median())
    mx = float(ts_m.max())
    base = max(1.0, m12, med, 1.1 * mx)
    cap = base * 6.0  # adjust 4.0–8.0 as needed
    return cap

# ---------- v5 additions: rate modelling & robust smoothing ----------

def monthly_rate_series(ts_m: pd.Series) -> Tuple[pd.Series, pd.Series]:
    """Return (rate_per_workday, workdays series aligned to ts_m)."""
    w = ts_m.index.to_series().apply(lambda p: business_days_in_month(p.start_time.year, p.start_time.month))
    w = w.astype(float).replace(0, np.nan)
    rate = ts_m / w
    return rate, w


def robust_roll_cap(series: pd.Series, window: int = 12, K: float = 6.0) -> pd.Series:
    """Apply rolling Median ± K*MAD cap to stabilize spikes without flattening the series."""
    s = series.copy().astype(float)
    vals = s.values
    for i in range(len(s)):
        lo = max(0, i - window)
        ref = vals[lo:i] if i > 0 else []
        if len(ref) >= 4:
            med = np.median(ref)
            mad = np.median(np.abs(ref - med)) + 1e-9
            upper = med + K * mad
            lower = max(0.0, med - K * mad)
            vals[i] = min(max(vals[i], lower), upper)
        else:
            vals[i] = max(vals[i], 0.0)
    return pd.Series(vals, index=s.index)



## 04. Month modelling (log-scale, rate-aware)

In [4]:
# ==================== Monthly modelling (log-scale, rate-aware) ====================

def fit_prophet_monthly_log(ts_m: pd.Series, is_rate: bool = False):
    """Fit Prophet on log1p(ts_m). If is_rate=True, ts_m is a rate series (per workday)."""
    if Prophet is None:
        return None, None
    y = np.log1p(ts_m.values)
    dfp = pd.DataFrame({'ds': ts_m.index.to_timestamp(), 'y': y})
    m = Prophet(weekly_seasonality=False, yearly_seasonality=True, daily_seasonality=False)
    m.fit(dfp)

    def fcast(h_months=H_MONTHS, future_workdays: Optional[pd.Series] = None):
        future = m.make_future_dataframe(periods=h_months, freq='MS')
        pred = m.predict(future)
        pred = pred.set_index(pd.PeriodIndex(pred['ds'], freq='M'))['yhat']
        pred = pred.iloc[-h_months:]
        # for rates: we do not cap in rate-space; just ensure numeric safety
        vals = expm1_safe(pred.values, cap_original=None if is_rate else compute_dynamic_cap(ts_m))
        if is_rate and future_workdays is not None:
            vals = vals * future_workdays.values
        return pd.Series(vals, index=pred.index)

    return m, fcast


def fit_arima_monthly_log(ts_m: pd.Series, is_rate: bool = False):
    """
    SARIMAX on log1p(ts_m) with a conservative grid. If is_rate=True, ts_m is rate series.
    Grid: p,q in {0,1}; d in {0,1} (prefer d=1 for short histories); P,Q in {0,1} if seasonal.
    """
    y = np.log1p(ts_m)
    best_aic, best_model = np.inf, None
    pqs = [0, 1]  # conservative
    seasonal = len(ts_m) >= 12
    PsQs = [0, 1] if seasonal else [0]

    for p in pqs:
        for d in ([1] if len(ts_m) < 36 else [0, 1]):
            for q in pqs:
                for P in PsQs:
                    for D in ([0, 1] if seasonal else [0]):
                        for Q in PsQs:
                            try:
                                model = SARIMAX(
                                    y, order=(p, d, q),
                                    seasonal_order=(P, D, Q, 12 if seasonal else 0),
                                    enforce_stationarity=False,
                                    enforce_invertibility=False
                                ).fit(disp=False)
                                if model.aic < best_aic:
                                    best_aic = model.aic
                                    best_model = model
                            except Exception:
                                continue

    def fcast(h_months=H_MONTHS, future_workdays: Optional[pd.Series] = None):
        fc_log = best_model.get_forecast(h_months).predicted_mean
        idx = pd.period_range(ts_m.index[-1] + 1, periods=h_months, freq='M')
        vals = expm1_safe(fc_log, cap_original=None if is_rate else compute_dynamic_cap(ts_m))
        if is_rate and future_workdays is not None:
            vals = vals * future_workdays.values
        return pd.Series(vals, index=idx)

    return best_model, fcast


def fit_tbats_or_ets_monthly_log(ts_m: pd.Series, is_rate: bool = False):
    """
    TBATS on log1p(ts_m) if available; else ETS(log1p). If is_rate=True, ts_m is rate.
    """
    y_log = np.log1p(ts_m)

    if TBATS is not None and len(ts_m) >= 12:
        y_log_ts = pd.Series(y_log.values, index=ts_m.index.to_timestamp())
        estimator = TBATS(use_arma_errors=False, seasonal_periods=[12])
        model = estimator.fit(y_log_ts)

        def fcast(h_months=H_MONTHS, future_workdays: Optional[pd.Series] = None):
            vals_log = model.forecast(steps=h_months)
            idx = pd.period_range(ts_m.index[-1] + 1, periods=h_months, freq='M')
            vals = expm1_safe(vals_log, cap_original=None if is_rate else compute_dynamic_cap(ts_m))
            if is_rate and future_workdays is not None:
                vals = vals * future_workdays.values
            return pd.Series(vals, index=idx)

        return model, fcast

    else:
        seasonal = 12 if len(ts_m) >= 24 else None
        model = ExponentialSmoothing(y_log, trend='add',
                                     seasonal=('add' if seasonal else None),
                                     seasonal_periods=seasonal).fit()

        def fcast(h_months=H_MONTHS, future_workdays: Optional[pd.Series] = None):
            vals_log = model.forecast(h_months)
            idx = pd.period_range(ts_m.index[-1] + 1, periods=h_months, freq='M')
            vals = expm1_safe(vals_log, cap_original=None if is_rate else compute_dynamic_cap(ts_m))
            if is_rate and future_workdays is not None:
                vals = vals * future_workdays.values
            return pd.Series(vals, index=idx)

        return model, fcast


def fit_ets_damped_monthly_log(ts_m: pd.Series, is_rate: bool = False):
    """ETS with damped trend on log1p(ts_m); stable candidate for blending."""
    y = np.log1p(ts_m)
    seasonal = 12 if len(ts_m) >= 24 else None
    model = ExponentialSmoothing(y, trend='add', damped_trend=True,
                                 seasonal=('add' if seasonal else None),
                                 seasonal_periods=seasonal).fit()

    def fcast(h_months=H_MONTHS, future_workdays: Optional[pd.Series] = None):
        vals_log = model.forecast(h_months)
        idx = pd.period_range(ts_m.index[-1] + 1, periods=h_months, freq='M')
        vals = expm1_safe(vals_log, cap_original=None if is_rate else compute_dynamic_cap(ts_m))
        if is_rate and future_workdays is not None:
            vals = vals * future_workdays.values
        return pd.Series(vals, index=idx)

    return model, fcast

# ==================== Adaptive CV (rate-aware) ====================

def rolling_cv_monthly_adaptive_rate(ts_vol: pd.Series) -> Tuple[Optional[Dict[str, float]], Optional[Dict[str, float]]]:
    """
    Adaptive rolling-origin CV using rate modelling internally:
    - If n >= 15 -> h=3; If 9 <= n < 15 -> h=1
    Returns two dicts: (mean sMAPE per model, mean wMAPE per model).
    Non-finite predictions in a fold are penalized with 200 (worst-case).
    """
    n = len(ts_vol)
    if n < 9:
        return None, None
    h = 3 if n >= 15 else 1
    min_train = max(12, n - (h + 2))

    s_out, w_out = [], []
    for start in range(min_train, n - h + 1):
        train_vol = ts_vol.iloc[:start]
        test_vol = ts_vol.iloc[start:start + h]

        # Build rate for training and workdays for future h months (relative to the training end)
        train_rate, _ = monthly_rate_series(train_vol)
        future_idx = pd.period_range(train_vol.index[-1] + 1, periods=h, freq='M')
        future_w = future_idx.to_series().apply(lambda p: business_days_in_month(p.start_time.year, p.start_time.month)).astype(float)

        s_metrics, w_metrics = {}, {}

        # Prophet
        mp, fp = fit_prophet_monthly_log(train_rate, is_rate=True)
        if fp is not None:
            try:
                pred_vol = fp(h_months=h, future_workdays=future_w)
                pv = np.array(pred_vol.values[:h], dtype=float)
                pv[~np.isfinite(pv)] = np.nan
                s_metrics['Prophet'] = 200.0 if np.isnan(pv).all() else smape(test_vol.values, np.nan_to_num(pv, nan=0.0))
                w_metrics['Prophet'] = 200.0 if np.isnan(pv).all() else wmape(test_vol.values, np.nan_to_num(pv, nan=0.0))
            except Exception:
                s_metrics['Prophet'] = 200.0; w_metrics['Prophet'] = 200.0

        # ARIMA
        try:
            ma, fa = fit_arima_monthly_log(train_rate, is_rate=True)
            pred_vol = fa(h_months=h, future_workdays=future_w)
            pv = np.array(pred_vol.values[:h], dtype=float)
            pv[~np.isfinite(pv)] = np.nan
            s_metrics['ARIMA'] = 200.0 if np.isnan(pv).all() else smape(test_vol.values, np.nan_to_num(pv, nan=0.0))
            w_metrics['ARIMA'] = 200.0 if np.isnan(pv).all() else wmape(test_vol.values, np.nan_to_num(pv, nan=0.0))
        except Exception:
            s_metrics['ARIMA'] = 200.0; w_metrics['ARIMA'] = 200.0

        # TBATS/ETS
        try:
            mt, ft = fit_tbats_or_ets_monthly_log(train_rate, is_rate=True)
            pred_vol = ft(h_months=h, future_workdays=future_w)
            pv = np.array(pred_vol.values[:h], dtype=float)
            pv[~np.isfinite(pv)] = np.nan
            s_metrics['TBATS/ETS'] = 200.0 if np.isnan(pv).all() else smape(test_vol.values, np.nan_to_num(pv, nan=0.0))
            w_metrics['TBATS/ETS'] = 200.0 if np.isnan(pv).all() else wmape(test_vol.values, np.nan_to_num(pv, nan=0.0))
        except Exception:
            s_metrics['TBATS/ETS'] = 200.0; w_metrics['TBATS/ETS'] = 200.0

        # ETS Damped (for blending only; not displayed in CV sheet)
        try:
            me, fe = fit_ets_damped_monthly_log(train_rate, is_rate=True)
            pred_vol = fe(h_months=h, future_workdays=future_w)
            pv = np.array(pred_vol.values[:h], dtype=float)
            pv[~np.isfinite(pv)] = np.nan
            s_metrics['ETS_Damped'] = 200.0 if np.isnan(pv).all() else smape(test_vol.values, np.nan_to_num(pv, nan=0.0))
            w_metrics['ETS_Damped'] = 200.0 if np.isnan(pv).all() else wmape(test_vol.values, np.nan_to_num(pv, nan=0.0))
        except Exception:
            s_metrics['ETS_Damped'] = 200.0; w_metrics['ETS_Damped'] = 200.0

        s_out.append(s_metrics); w_out.append(w_metrics)

    sm = pd.DataFrame(s_out).mean().to_dict()
    wm = pd.DataFrame(w_out).mean().to_dict()
    return sm, wm

# ==================== Blending ====================

def select_or_blend_forecasts(fc_dict: Dict[str, pd.Series],
                              cv_scores_wmape: Dict[str, float],
                              blend: bool = True):
    """
    Blend using 1/wMAPE as weights (lower better). If blending not possible, select min wMAPE.
    """
    scores = {k: (v if v is not None and np.isfinite(v) else 1e6) for k, v in cv_scores_wmape.items()}
    # Filter fc_dict to models that have a score
    models = [m for m in fc_dict.keys() if m in scores]
    if not models:
        # fallback to any available key
        k0 = list(fc_dict.keys())[0]
        return fc_dict[k0], {'winner': k0, 'weights': {k0: 1.0}}

    if not blend:
        best = min(models, key=lambda m: scores[m])
        return fc_dict[best], {'winner': best, 'weights': {best: 1.0}}

    inv = {m: (1.0 / scores[m] if scores[m] > 0 else 0.0) for m in models}
    total = sum(inv.values())
    if total == 0:
        best = min(models, key=lambda m: scores[m])
        return fc_dict[best], {'winner': best, 'weights': {best: 1.0}}
    w = {m: inv[m] / total for m in models}

    # Align indices and blend
    idx = None
    for s in fc_dict.values():
        idx = s.index if idx is None else idx.union(s.index)
    blended = sum(w[m] * fc_dict[m].reindex(idx).fillna(0) for m in models)
    return blended, {'winner': min(models, key=lambda m: scores[m]), 'weights': w}



## 05. Monthly pipeline

In [5]:
def build_monthly_series(df: pd.DataFrame) -> pd.DataFrame:
    """Aggregate daily incoming to monthly by department."""
    df = df.copy()
    df['month'] = df['Date'].dt.to_period('M')
    monthly = (df.groupby(['department_id', 'month'], as_index=False)['ticket_total']
               .sum()
               .rename(columns={'ticket_total': 'incoming_monthly'}))
    return monthly


def forecast_per_department_monthly(monthly: pd.DataFrame) -> pd.DataFrame:
    """
    Hybrid modelling per department (rate-aware) + adaptive CV + robust sanitation.
    Returns columns for capacity_error and mape_table_cv.
    """
    out_rows = []
    dept_ids = monthly['department_id'].unique().tolist()

    for dept in dept_ids:
        ts_vol = (monthly.loc[monthly['department_id'] == dept, ['month', 'incoming_monthly']]
                  .sort_values('month')
                  .set_index('month')['incoming_monthly'])
        if not pd.api.types.is_period_dtype(ts_vol.index):
            ts_vol.index = pd.PeriodIndex(ts_vol.index, freq='M')
        if len(ts_vol) == 0:
            continue

        # Winsorize volumes and build rate series
        ts_vol = winsorize_monthly(ts_vol, 0.01, 0.99)
        ts_rate, _ = monthly_rate_series(ts_vol)
        ts_rate = ts_rate.fillna(ts_rate.median()).clip(lower=0)

        # Future workdays index
        future_idx = pd.period_range(ts_vol.index[-1] + 1, periods=H_MONTHS, freq='M')
        future_w = future_idx.to_series().apply(lambda p: business_days_in_month(p.start_time.year, p.start_time.month)).astype(float)

        cv_smape, cv_wmape = {}, {}

        # Collect forecasts (on volume scale returned by each fcast with is_rate=True)
        fc_dict: Dict[str, pd.Series] = {}

        # Prophet
        if Prophet is not None and len(ts_rate) >= 12:
            try:
                _, fp = fit_prophet_monthly_log(ts_rate, is_rate=True)
                if fp is not None:
                    fc_dict['Prophet'] = fp(H_MONTHS, future_workdays=future_w)
            except Exception:
                pass

        # ARIMA
        try:
            _, fa = fit_arima_monthly_log(ts_rate, is_rate=True)
            fc_dict['ARIMA'] = fa(H_MONTHS, future_workdays=future_w)
        except Exception:
            pass

        # TBATS/ETS
        try:
            _, ft = fit_tbats_or_ets_monthly_log(ts_rate, is_rate=True)
            fc_dict['TBATS/ETS'] = ft(H_MONTHS, future_workdays=future_w)
        except Exception:
            pass

        # ETS Damped (candidate for blending & CV; not shown in CV sheet columns)
        try:
            _, fe = fit_ets_damped_monthly_log(ts_rate, is_rate=True)
            fc_dict['ETS_Damped'] = fe(H_MONTHS, future_workdays=future_w)
        except Exception:
            pass

        if not fc_dict:
            idx = pd.period_range(ts_vol.index[-1] + 1, periods=H_MONTHS, freq='M')
            val = max(0.0, float(ts_vol.mean()))
            fc_dict['NaiveMean'] = pd.Series([val] * H_MONTHS, index=idx)

        # CV metrics (mean) using rate-aware CV
        try:
            cv_smape, cv_wmape = rolling_cv_monthly_adaptive_rate(ts_vol)
            cv_smape = cv_smape or {}
            cv_wmape = cv_wmape or {}
        except Exception:
            cv_smape, cv_wmape = {}, {}

        # Blend/select using wMAPE
        blended, meta = select_or_blend_forecasts(fc_dict, cv_scores_wmape=cv_wmape, blend=True)

        # Enforce finiteness
        if not np.isfinite(blended.values).all():
            finite_mask = np.isfinite(blended.values)
            if finite_mask.any():
                finite_mean = float(np.nanmean(blended.values[finite_mask]))
                vals = np.where(finite_mask, blended.values, finite_mean)
                blended = pd.Series(vals, index=blended.index)
            else:
                idx = pd.period_range(ts_vol.index[-1] + 1, periods=H_MONTHS, freq='M')
                val = max(0.0, float(ts_vol.mean()))
                blended = pd.Series([val] * H_MONTHS, index=idx)

        # Robust smoothing (Median ± K·MAD) to kill residual spikes
        blended = robust_roll_cap(blended, window=12, K=6.0)

        # Optional growth guard vs local ref
        if APPLY_LOCAL_GROWTH_GUARD:
            ref = max(1.0, float(ts_vol.tail(12).mean())) if len(ts_vol) else 1.0
            blended = blended.clip(lower=ref * MIN_GROWTH, upper=ref * MAX_GROWTH)

        # Prepare output rows (store sMAPE CV columns; weights from wMAPE-based blend)
        # Extract weights for sheet columns (Prophet/ARIMA/TBATS/ETS only)
        w_prophet = meta['weights'].get('Prophet', np.nan)
        w_arima = meta['weights'].get('ARIMA', np.nan)
        w_tbats = meta['weights'].get('TBATS/ETS', np.nan)

        for per, val in blended.items():
            out_rows.append({
                'department_id': dept,
                'month': per,
                'forecast_monthly': max(0.0, float(val)),
                'cv_prophet_smape': cv_smape.get('Prophet', np.nan),
                'cv_arima_smape': cv_smape.get('ARIMA', np.nan),
                'cv_tbats_ets_smape': cv_smape.get('TBATS/ETS', np.nan),
                'winner_model': meta['winner'],
                'blend_prophet_w': w_prophet,
                'blend_arima_w': w_arima,
                'blend_tbats_ets_w': w_tbats,
            })

    df_out = pd.DataFrame(out_rows)
    if not df_out.empty:
        df_out['department_id'] = df_out['department_id'].astype(str)
        if not pd.api.types.is_period_dtype(df_out['month']):
            df_out['month'] = pd.PeriodIndex(df_out['month'], freq='M')
    return df_out


def compute_monthly_accuracy_with_history(monthly: pd.DataFrame,
                                          fc_monthly: pd.DataFrame,
                                          report_start: str) -> pd.DataFrame:
    """Build capacity_error-like table with historical Actuals and future Forecasts."""
    monthly = monthly.copy()
    monthly['department_id'] = monthly['department_id'].astype(str)
    if not pd.api.types.is_period_dtype(monthly['month']):
        monthly['month'] = pd.PeriodIndex(monthly['month'], freq='M')

    fc = fc_monthly.copy()
    fc['department_id'] = fc['department_id'].astype(str)
    if not pd.api.types.is_period_dtype(fc['month']):
        fc['month'] = pd.PeriodIndex(fc['month'], freq='M')

    start_per = pd.Period(report_start, freq='M')
    last_actual = monthly['month'].max()

    hist = (monthly.loc[monthly['month'] >= start_per, ['department_id', 'month', 'incoming_monthly']]
            .rename(columns={'incoming_monthly': 'Actual_Volume'}))
    hist['Forecast'] = np.nan

    fut = fc[['department_id', 'month', 'forecast_monthly',
              'cv_prophet_smape', 'cv_arima_smape', 'cv_tbats_ets_smape',
              'winner_model', 'blend_prophet_w', 'blend_arima_w', 'blend_tbats_ets_w']].copy()
    fut = fut.loc[fut['month'] > last_actual]
    fut = fut.rename(columns={'forecast_monthly': 'Forecast'})
    fut['Actual_Volume'] = np.nan

    base = pd.concat([hist, fut], ignore_index=True, sort=False)

    base['Forecast_Accuracy'] = np.where(
        (base['Actual_Volume'].notna()) & (base['Forecast'].notna()) & (base['Actual_Volume'] > 0),
        (1 - (np.abs(base['Forecast'] - base['Actual_Volume']) / base['Actual_Volume'])) * 100.0,
        np.nan
    )
    return base


def compute_capacity_monthly(cap_df: pd.DataFrame, prod_dept: pd.DataFrame) -> pd.DataFrame:
    """Compute FTE/day needed per month = Forecast / (avg_tickets_per_agent_day * workdays_in_month)."""
    out = cap_df.merge(prod_dept, on='department_id', how='left')
    out['avg_tickets_per_agent_day'] = pd.to_numeric(out['avg_tickets_per_agent_day'], errors='coerce')
    out['avg_tickets_per_agent_day'] = out['avg_tickets_per_agent_day'].replace(0, np.nan)
    out['workdays_in_month'] = [business_days_in_month(m.start_time.year, m.start_time.month) for m in out['month']]
    out['Capacity_FTE_per_day'] = np.where(
        (out['avg_tickets_per_agent_day'] > 0) & (out['workdays_in_month'] > 0) & (out['Forecast'].notna()),
        out['Forecast'] / (out['avg_tickets_per_agent_day'] * out['workdays_in_month']),
        np.nan
    )
    return out


def build_cv_table(fc_monthly: pd.DataFrame, mapping: pd.DataFrame) -> pd.DataFrame:
    """Build mape_table_cv with sMAPE, best model and weights (from wMAPE-based blend)."""
    if fc_monthly is None or fc_monthly.empty:
        raise ValueError("fc_monthly is empty; cannot build CV table.")
    cols_keep = [
        'department_id',
        'cv_prophet_smape', 'cv_arima_smape', 'cv_tbats_ets_smape',
        'winner_model',
        'blend_prophet_w', 'blend_arima_w', 'blend_tbats_ets_w'
    ]
    df = (fc_monthly[cols_keep]
          .drop_duplicates(subset=['department_id'])
          .copy())
    df = df.rename(columns={
        'cv_prophet_smape': 'sMAPE_Prophet_CV',
        'cv_arima_smape': 'sMAPE_ARIMA_CV',
        'cv_tbats_ets_smape': 'sMAPE_TBATS_ETS_CV',
        'winner_model': 'Best_Model',
        'blend_prophet_w': 'Weight_Prophet',
        'blend_arima_w': 'Weight_ARIMA',
        'blend_tbats_ets_w': 'Weight_TBATS_ETS',
    })
    df['department_id'] = df['department_id'].astype(str)
    df = apply_mapping(df, mapping)
    ordered_cols = [
        'department_id', 'department_name', 'vertical',
        'sMAPE_Prophet_CV', 'sMAPE_ARIMA_CV', 'sMAPE_TBATS_ETS_CV',
        'Best_Model',
        'Weight_Prophet', 'Weight_ARIMA', 'Weight_TBATS_ETS'
    ]
    df = df[ordered_cols]
    return df.sort_values(['vertical', 'department_id'])


## 06. Daily pipeline (reconciled with monthly)

In [6]:
def dow_profile(g: pd.DataFrame) -> pd.Series:
    """Build normalized day-of-week profile for a department, fallback to uniform."""
    prof = (g.assign(dow=g['Date'].dt.dayofweek)
              .groupby('dow')['ticket_total']
              .mean())
    if prof.notna().sum() >= 3:
        prof = prof / prof.mean()
    else:
        prof = pd.Series(1.0, index=range(7))
    return prof


def disaggregate_month_to_days(dept_df: pd.DataFrame,
                               month_period: pd.Period,
                               target_sum: float) -> pd.DataFrame:
    """
    Allocate monthly forecast to each day in that month using recent DOW profile,
    guaranteeing that daily sum equals the monthly target (within rounding).
    """
    start = month_period.start_time
    end = month_period.end_time
    days = pd.date_range(start=start, end=end, freq='D')

    hist = dept_df.sort_values('Date').tail(90)
    profile = dow_profile(hist)

    weights = np.array([profile.get(d.dayofweek, 1.0) for d in days], dtype=float)
    weights = np.maximum(weights, 1e-6)
    weights = weights / weights.sum()

    alloc = target_sum * weights
    return pd.DataFrame({'Date': days, 'forecast_daily': alloc})


def build_daily_from_monthly(incoming: pd.DataFrame,
                             fc_monthly: pd.DataFrame,
                             horizon_days: int) -> pd.DataFrame:
    """
    Top-down daily plan: disaggregate monthly forecast using DOW profile for days within horizon.
    """
    last_date = incoming['Date'].max()
    start = last_date + pd.Timedelta(days=1)
    end = start + pd.Timedelta(days=horizon_days - 1)
    future_months = pd.period_range(start=start.to_period('M'),
                                    end=end.to_period('M'), freq='M')

    rows = []
    for dept, g in incoming.groupby('department_id'):
        for m in future_months:
            fcm = fc_monthly[(fc_monthly['department_id'] == dept) & (fc_monthly['month'] == m)]
            if fcm.empty:
                continue
            target = float(fcm['forecast_monthly'].iloc[0])
            if target <= 0:
                continue
            alloc_df = disaggregate_month_to_days(g, m, target)
            alloc_df = alloc_df[(alloc_df['Date'] >= start) & (alloc_df['Date'] <= end)]
            alloc_df.insert(0, 'department_id', dept)
            rows.append(alloc_df)

    df = pd.concat(rows, ignore_index=True) if rows else pd.DataFrame(columns=['department_id', 'Date', 'forecast_daily'])
    return df


def split_daily_by_language(df_daily_fc: pd.DataFrame) -> pd.DataFrame:
    """Split daily forecast by fixed language shares."""
    parts = []
    for lang, w in LANGUAGE_SHARES.items():
        tmp = df_daily_fc.copy()
        tmp['language'] = lang
        tmp['forecast_daily_language'] = tmp['forecast_daily'] * w
        parts.append(tmp)
    out = pd.concat(parts, ignore_index=True) if parts else pd.DataFrame()
    return out


def forecast_daily_baseline(df_daily: pd.DataFrame, horizon_days: int) -> pd.DataFrame:
    """Independent daily baseline (kept as optional)."""
    df = df_daily.copy()
    df['Date'] = pd.to_datetime(df['Date'])
    df['department_id'] = df['department_id'].astype(str).str.strip()
    df = df.sort_values(['department_id', 'Date'])
    last_date = df['Date'].max()
    if pd.isna(last_date):
        raise ValueError("forecast_daily_baseline: No valid dates in incoming.")
    start = last_date + pd.Timedelta(days=1)
    idx_future = pd.date_range(start=start, periods=horizon_days, freq='D')

    rows = []
    for dept, g in df.groupby('department_id'):
        g = g.sort_values('Date')
        if len(g) >= 28:
            roll_mean = (g.set_index('Date')['ticket_total']
                         .rolling(window=28, min_periods=1)
                         .mean()
                         .iloc[-1])
            base = float(roll_mean) if np.isfinite(roll_mean) else float(g['ticket_total'].mean())
        else:
            base = float(g['ticket_total'].mean())

        prof = dow_profile(g)
        vals = []
        for d in idx_future:
            w = prof[d.dayofweek] if d.dayofweek in prof.index else 1.0
            vals.append(max(0.0, base * float(w)))
        rows.append(pd.DataFrame({'department_id': dept, 'Date': idx_future, 'forecast_daily': vals}))

    return pd.concat(rows, ignore_index=True) if rows else pd.DataFrame(columns=['department_id', 'Date', 'forecast_daily'])


def build_daily_capacity_plan(incoming: pd.DataFrame,
                              mapping: pd.DataFrame,
                              prod_dept: pd.DataFrame,
                              fc_monthly: pd.DataFrame,
                              horizon_days: int) -> pd.DataFrame:
    """
    End-to-end daily plan:
    - If USE_DAILY_FROM_MONTHLY: disaggregate monthly forecast (reconciled)
    - Else: robust daily baseline (28D moving average)
    - Split by languages
    - Attach department_name / vertical
    - Compute FTE per day per department/language
    """
    if USE_DAILY_FROM_MONTHLY:
        daily_fc = build_daily_from_monthly(incoming, fc_monthly, horizon_days)
    else:
        daily_fc = forecast_daily_baseline(incoming, horizon_days)

    daily_fc_lang = split_daily_by_language(daily_fc)

    # Attach names/vertical
    daily_fc_lang = apply_mapping(daily_fc_lang, mapping)

    # Merge productivity
    daily_fc_lang = daily_fc_lang.merge(prod_dept, on='department_id', how='left')

    # Compute FTE requirement per day
    daily_fc_lang['avg_tickets_per_agent_day'] = pd.to_numeric(daily_fc_lang['avg_tickets_per_agent_day'], errors='coerce')
    daily_fc_lang['FTE_per_day'] = np.where(
        daily_fc_lang['avg_tickets_per_agent_day'] > 0,
        daily_fc_lang['forecast_daily_language'] / daily_fc_lang['avg_tickets_per_agent_day'],
        np.nan
    )
    cols = ['Date', 'department_id', 'department_name', 'vertical', 'language',
            'forecast_daily_language', 'FTE_per_day']
    daily_plan = daily_fc_lang[cols].sort_values(['Date', 'vertical', 'department_id', 'language'])
    return daily_plan


## 07. Main & Excel writing

In [7]:
def main():
    # 1) Load inputs
    incoming = load_incoming(INCOMING_SOURCE_PATH, sheet_name=INCOMING_SHEET)
    mapping = load_dept_map(DEPT_MAP_PATH, DEPT_MAP_SHEET)
    prod = load_productivity(PRODUCTIVITY_PATH)

    # 2) Monthly forecast (rate-aware)
    monthly = build_monthly_series(incoming)
    fc_monthly = forecast_per_department_monthly(monthly)

    # 3) capacity_error (historicals + future forecast)
    cap_err = compute_monthly_accuracy_with_history(monthly, fc_monthly, REPORT_START_MONTH)
    cap_err = compute_capacity_monthly(cap_err, prod)
    cap_err = apply_mapping(cap_err, mapping)

    # 4) Daily plan (reconciled with monthly by default)
    daily_capacity_plan = build_daily_capacity_plan(incoming, mapping, prod, fc_monthly, DAILY_HORIZON_DAYS)

    # 5) CV table
    cv_table = build_cv_table(fc_monthly, mapping)

    # 6) Ensure no inf/-inf propagate to Excel
    for df_out in [cap_err, daily_capacity_plan, cv_table]:
        df_out.replace([np.inf, -np.inf], np.nan, inplace=True)

    # 7) Write Excel with required sheet names
    with pd.ExcelWriter(OUTPUT_XLSX, engine="openpyxl", mode="w") as w:
        (cap_err[['vertical', 'department_id', 'department_name', 'month',
                  'Actual_Volume', 'Forecast', 'Forecast_Accuracy',
                  'Capacity_FTE_per_day',
                  'winner_model', 'cv_prophet_smape', 'cv_arima_smape', 'cv_tbats_ets_smape',
                  'blend_prophet_w', 'blend_arima_w', 'blend_tbats_ets_w']]
         .sort_values(['vertical', 'department_id', 'month'])
         .to_excel(w, "capacity_error", index=False))

        daily_capacity_plan.to_excel(w, "daily_capacity_plan", index=False)
        cv_table.to_excel(w, "mape_table_cv", index=False)

    print("Excel written:", OUTPUT_XLSX)


if __name__ == "__main__":
    main()

18:33:18 - cmdstanpy - INFO - Chain [1] start processing
18:33:25 - cmdstanpy - INFO - Chain [1] done processing
18:33:42 - cmdstanpy - INFO - Chain [1] start processing
18:33:49 - cmdstanpy - INFO - Chain [1] done processing
18:34:12 - cmdstanpy - INFO - Chain [1] start processing
18:34:16 - cmdstanpy - INFO - Chain [1] done processing
18:34:33 - cmdstanpy - INFO - Chain [1] start processing
18:34:34 - cmdstanpy - INFO - Chain [1] done processing
18:34:54 - cmdstanpy - INFO - Chain [1] start processing
18:34:54 - cmdstanpy - INFO - Chain [1] done processing
18:35:13 - cmdstanpy - INFO - Chain [1] start processing
18:35:21 - cmdstanpy - INFO - Chain [1] done processing
18:35:46 - cmdstanpy - INFO - Chain [1] start processing
18:35:50 - cmdstanpy - INFO - Chain [1] done processing
18:36:13 - cmdstanpy - INFO - Chain [1] start processing
18:36:14 - cmdstanpy - INFO - Chain [1] done processing
18:37:28 - cmdstanpy - INFO - Chain [1] start processing
18:37:37 - cmdstanpy - INFO - Chain [1]

Excel written: C:\Users\pt3canro\Desktop\CAPACITY\outputs\capacity_forecast_hybrid.xlsx
